In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Formas de caracterizar en contexto

Haremos los siguientes experimentos para ver la mejor forma de caracterizar el contexto:
    - bolsa de palabras (unigramas, bigramas, subwords, con y sin stopwords) 
    - Agregado por mi: tfidf con normalización para poner algunos pesos en las palabras
    - si hay interacción del usuario objetivo y cómo se representa
    - si hay respuestas
    - si hay multimedia y de qué tipo
__COMPLETAR__

__PREGUNTAR:
si hay interacción del usuario objetivo y cómo se representa??. si hay respuestas??__

Usaremos diferentes word embeddings, obtenidos de diferentes tipos de corpus: genéricos o específicos de lenguaje generado por usuario o específicos del corpus del que estamos aprendiendo.
    - Es decir, tres diferentes tipos de word embeddings:
      - SBWCE
      - con el mismo corpus de chats
      - con un corpus de twitter
      
__BUSCAR CORPUS DE TWITTER__

## Cargando los datos de entrenamiento

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('short_ans3.csv')

Acordandonos de los datos..

In [4]:
sum(df.context.isnull())

1

In [5]:
df

,idchat,context,label
0,0,NaN,https://m.facebook.com/story.php?story_fbid=14...
1,0,https://m.facebook.com/story.php?story_fbid=14...,nonDef
2,1,😂😂😂😂😂😂😂😂😂😂. Que es este lugar 👀. Qur hace este...,Bienvenido
3,1,Bienvenido. Vengo a pasarle fotos obscenas e i...,👀
4,1,👀. El mundo de nunca jamas?),😂😂
5,1,😂😂,😂😂😂
6,1,😂😂😂,Oh sii
7,1,Oh sii. Seremos niños por siempre,Quienes ? 👀😂
8,1,Quienes ? 👀😂,Yo
9,1,Yo. 😏,😏


# Pre procesamiento

Como habíamos dicho es mejor pre procesar las palabras

In [6]:
from string import punctuation
import re
import unicodedata

In [7]:
STOP_WORDS = punctuation + '¡' + '¿'

def dig2num(text):
    return re.sub('(\d+[\WxX]*[^ ]*)+', 'NUM', text)

def rm_tildes(text):
    return ''.join((c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'))

def rm_media(text):
    return re.sub(r'<archivo omitido>|<media omitted>', 'MEDIA', text)

def rm_repeated(text):
    return re.sub(r'([aeourshmi])\1{2,}', r'\1', text)

def rm_ok(text):
    return re.sub('(ok)+[akis]*', 'ok', text)

def rm_sw(text):
    return re.sub('{1}{0}{1}'.format(STOP_WORDS, '[]'), '', text)

def rm_ja(text):
    return re.sub('(ja[ja(js)]*)+', 'ja', text)

def rm_si(text):
    return re.sub('[s]+[i]+', 'si', text)

def to_low(text):
    return text.lower()

def clean_text(text):
    return rm_media(rm_tildes(to_low(text)))

def clean_text_1(text):
    return dig2num(rm_si(rm_ok(rm_ja(rm_repeated(rm_sw(clean_text(text)))))))

In [8]:
df_cleaned = df[1:].context.map(clean_text_1)
df_cleaned.head(30)

1     httpsmfacebookcomstoryphpstoryfbidNUM hola te ...
2     😂😂😂😂😂😂😂😂😂😂 que es este lugar 👀 qur hace este a...
3     bienvenido vengo a pasarle fotos obscenas e ir...
4                             👀 el mundo de nunca jamas
5                                                    😂😂
6                                                   😂😂😂
7                       oh si seremos ninos por siempre
8                                           quienes  👀😂
9                                                  yo 😏
10                                                    😏
11                                                    😏
12     😏 ja eso si que es bastante creativo muy bueno 😄
13                                                    😎
14                                                  🌚 🌚
15                                                    😏
16                                                 hola
17    hola 🇻🇪 que yo me desaparezca no hay lio pero ...
18    😱😱 quien ja mira que adulador es nuestro m

## Vectorizando

#### BOG: 1-gramas

In [65]:
vectorizer = CountVectorizer(min_df=10, preprocessor=clean_text_1, ngram_range=(1, 1))

In [66]:
representation = vectorizer.fit(df[1:].context)

Entonces obtenemos alrededor de tantas features:

In [67]:
print(representation.get_feature_names()[:100])
len(representation.get_feature_names())

['MEDIA', 'NUM', 'aNUM', 'aa', 'aah', 'aaja', 'abajo', 'abi', 'abierta', 'abiertas', 'abierto', 'abra', 'abran', 'abrazo', 'abrazos', 'abre', 'abren', 'abri', 'abril', 'abrir', 'abro', 'abuela', 'abuelo', 'aburrido', 'abuso', 'aca', 'acaba', 'acaban', 'acabar', 'acabaron', 'acabas', 'acabe', 'acabo', 'academia', 'acaso', 'acceder', 'acceso', 'accion', 'acepta', 'acepte', 'acepto', 'acerca', 'aclarar', 'aclaro', 'acompana', 'acompanar', 'acompano', 'acordaba', 'acordar', 'acordas', 'acordate', 'acorde', 'actitud', 'activa', 'activan', 'activar', 'active', 'actividad', 'actividades', 'activo', 'activos', 'acto', 'actores', 'actual', 'actualiza', 'actualizacion', 'actualizar', 'actualizo', 'acuerda', 'acuerdan', 'acuerdo', 'acv', 'add', 'adelantar', 'adelante', 'adelanto', 'ademas', 'adentro', 'adios', 'adjuntado', 'admin', 'administrador', 'adri', 'adrian', 'aeropuerto', 'afecta', 'afiche', 'after', 'afuera', 'again', 'agarra', 'agarrar', 'agarre', 'agarro', 'age', 'agenda', 'agendado', 

5394

In [68]:
BOG = representation.transform(df[1:].context)

In [69]:
print(BOG[:10])

  (0, 1274)	1
  (0, 2425)	1
  (0, 2458)	1
  (0, 3442)	1
  (0, 4807)	1
  (1, 297)	1
  (1, 1766)	1
  (1, 1855)	2
  (1, 2320)	1
  (1, 2954)	1
  (1, 4164)	1
  (1, 4222)	1
  (2, 517)	1
  (2, 2094)	1
  (2, 2621)	1
  (2, 3672)	1
  (2, 5193)	1
  (3, 1274)	1
  (3, 1613)	1
  (3, 2643)	1
  (3, 3299)	1
  (3, 3444)	1
  (6, 3381)	1
  (6, 3473)	1
  (6, 3939)	1
  (6, 4608)	1
  (6, 4610)	1
  (7, 4203)	1
  (8, 5379)	1


#### BOG: 2-gramas

In [70]:
representation.set_params(ngram_range=(1, 2))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 2),
        preprocessor=<function clean_text_1 at 0x7fafab945c80>,
        stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)

In [71]:
len(representation.fit(df[1:].context).get_feature_names())

14403

#### BOG: 3-gramas

In [72]:
representation.set_params(ngram_range=(1, 3))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 3),
        preprocessor=<function clean_text_1 at 0x7fafab945c80>,
        stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)

In [73]:
len(representation.fit(df[1:].context).get_feature_names())

17048

In [74]:
BOG_matrix = representation.transform(df[1:].context)

In [75]:
BOG_matrix.get_shape()

(56778, 17048)

In [76]:
df.context[28]

'chicos. una pregunta. cómo se escribe la dirección postal en vuestras ciudades?. es que los otros días vi una de francia y es un poco diferente a como yo escribo la mía. La de aca por donde vivo es 1010. El cod postal. así que si pongo n un sobre 1010 te llega?. Pero es por zona, no se si s igual por alla. qué raro sois los venezolanos. no?. No, obvio pones la direccion😂😂'

In [77]:
arr_repr = BOG_matrix[27].toarray()[0]
print('{ ', end='')
for i in BOG_matrix[27].indices:
    print('{0}: {1}'.format(representation.get_feature_names()[i], arr_repr[i]), end=', ')
print('}', end='')

{ NUM: 2, aca: 1, aca por: 1, alla: 1, asi: 1, asi que: 1, asi que si: 1, chicos: 1, chicos una: 1, ciudades: 1, cod: 1, como: 2, como se: 1, como yo: 1, de: 2, de aca: 1, dias: 1, diferente: 1, direccion: 2, donde: 1, en: 1, es: 4, es NUM: 1, es por: 1, es que: 1, es un: 1, es un poco: 1, escribe: 1, escribo: 1, igual: 1, igual por: 1, la: 4, la de: 1, la direccion: 2, la mia: 1, llega: 1, los: 2, los otros: 1, mia: 1, no: 3, no no: 1, no se: 1, no se si: 1, obvio: 1, otros: 1, pero: 1, pero es: 1, poco: 1, pones: 1, pongo: 1, por: 3, por alla: 1, por donde: 1, pregunta: 1, que: 3, que los: 1, que raro: 1, que si: 1, raro: 1, se: 2, se escribe: 1, se si: 1, si: 2, si igual: 1, sobre: 1, un: 2, un poco: 1, una: 2, una de: 1, una pregunta: 1, vi: 1, vivo: 1, yo: 1, zona: 1, }

#### BOG: (3,4)-Subwords

In [78]:
vectorizer = CountVectorizer(min_df=10, stop_words=STOP_WORDS, preprocessor=clean_text_1, analyzer='char', ngram_range=(3,4))

In [79]:
vectorizer

CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(3, 4),
        preprocessor=<function clean_text_1 at 0x7fafab945c80>,
        stop_words='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¡',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [80]:
representation = vectorizer.fit(df[1:].context)
BOG_matrix = representation.transform(df[1:].context)

In [81]:
len(representation.get_feature_names())

24863

In [82]:
df.context[28]

'chicos. una pregunta. cómo se escribe la dirección postal en vuestras ciudades?. es que los otros días vi una de francia y es un poco diferente a como yo escribo la mía. La de aca por donde vivo es 1010. El cod postal. así que si pongo n un sobre 1010 te llega?. Pero es por zona, no se si s igual por alla. qué raro sois los venezolanos. no?. No, obvio pones la direccion😂😂'

In [83]:
arr_repr = BOG_matrix[27].toarray()[0]
for i in BOG_matrix[27].indices:
    print('{0}: {1}'.format(representation.get_feature_names()[i], arr_repr[i]), end=', ')

 NU: 2,  NUM: 2,  a : 1,  a c: 1,  ac: 1,  aca: 1,  al: 1,  all: 1,  as: 1,  asi: 1,  ci: 1,  ciu: 1,  co: 3,  cod: 1,  com: 2,  de: 2,  de : 2,  di: 4,  dia: 1,  dif: 1,  dir: 2,  do: 1,  don: 1,  en: 1,  en : 1,  es: 6,  es : 4,  esc: 2,  fr: 1,  fra: 1,  ig: 1,  igu: 1,  la: 4,  la : 4,  ll: 1,  lle: 1,  lo: 2,  los: 2,  mi: 1,  mia: 1,  n : 1,  no: 3,  no : 3,  ob: 1,  obv: 1,  ot: 1,  otr: 1,  pe: 1,  per: 1,  po: 8,  poc: 1,  pon: 2,  por: 3,  pos: 2,  pr: 1,  pre: 1,  qu: 3,  que: 3,  ra: 1,  rar: 1,  s : 1,  se: 2,  se : 2,  si: 2,  si : 2,  so: 2,  sob: 1,  un: 4,  un : 2,  una: 2,  ve: 1,  ven: 1,  vi: 2,  vi : 1,  viv: 1,  vu: 1,  vue: 1,  y : 1,  y e: 1,  yo: 1,  yo : 1,  zo: 1,  zon: 1, M c: 1, M co: 1, M l: 1, M ll: 1, NUM: 2, NUM : 2, UM : 2, UM c: 1, UM l: 1, a c: 2, a co: 2, a d: 4, a de: 2, a di: 2, a l: 1, a la: 1, a m: 1, a mi: 1, a n: 1, a no: 1, a p: 3, a pe: 1, a po: 1, a pr: 1, a q: 1, a qu: 1, a y: 1, a y : 1, aca: 1, aca : 1, ade: 1, ades: 1, al : 3, al a: 1, 

### TFidf: 1-gramas

In [84]:
vectorizer = TfidfVectorizer(min_df=10, preprocessor=clean_text_1, ngram_range=(1, 1), norm='l2')

In [85]:
representation = vectorizer.fit(df[1:].context)

In [86]:
BOG_matrix = representation.transform(df[1:].context)

Entonces podemos decir que la frase:

In [102]:
df.context[7]

'Oh sii. Seremos niños por siempre'

Esta representada como:

In [88]:
print(BOG_matrix[6])

  (0, 4610)	0.444007229186
  (0, 4608)	0.232292394347
  (0, 3939)	0.285705626552
  (0, 3473)	0.497750374311
  (0, 3381)	0.647699532027


Donde los features son...

In [89]:
representation.get_feature_names()[4449], representation.get_feature_names()[4447], representation.get_feature_names()[3802], representation.get_feature_names()[3348], representation.get_feature_names()[3261]

('saca', 'sabor', 'peso', 'necesiten', 'moral')

### TFidf: 3-gramas

In [90]:
vectorizer = TfidfVectorizer(min_df=10, preprocessor=clean_text_1, ngram_range=(1, 3), norm='l2')

In [91]:
representation = vectorizer.fit(df[1:].context)

In [92]:
BOG_matrix = representation.transform(df[1:].context)

In [93]:
print(BOG_matrix[:10])

  (0, 14962)	0.251093106197
  (0, 10991)	0.526693785555
  (0, 7090)	0.708823214338
  (0, 6946)	0.34980486296
  (0, 3044)	0.186419196821
  (1, 13416)	0.445651009621
  (1, 12802)	0.260870206023
  (1, 12700)	0.114103432201
  (1, 9129)	0.312658204155
  (1, 6600)	0.238068586996
  (1, 5707)	0.481824210078
  (1, 5161)	0.445651009621
  (1, 5101)	0.145516231116
  (1, 1117)	0.337573917747
  (2, 16328)	0.45418585109
  (2, 11596)	0.520149510929
  (2, 7360)	0.463337136898
  (2, 6171)	0.360458478114
  (2, 1504)	0.422549502516
  (3, 10995)	0.391758851566
  (3, 10264)	0.471306759228
  (3, 7671)	0.549778867213
  (3, 4420)	0.514157183574
  (3, 4180)	0.175496199971
  (3, 3044)	0.164260806241
  (6, 14494)	0.354404835023
  (6, 14225)	0.185414881299
  (6, 12162)	0.22804911449
  (6, 11046)	0.602397708492
  (6, 11040)	0.397302403417
  (6, 10463)	0.51699123506
  (7, 13366)	1.0
  (8, 16875)	1.0


Entonces podemos decir que la frase:

In [94]:
df.context[28]

'chicos. una pregunta. cómo se escribe la dirección postal en vuestras ciudades?. es que los otros días vi una de francia y es un poco diferente a como yo escribo la mía. La de aca por donde vivo es 1010. El cod postal. así que si pongo n un sobre 1010 te llega?. Pero es por zona, no se si s igual por alla. qué raro sois los venezolanos. no?. No, obvio pones la direccion😂😂'

Esta representada como:

In [95]:
print(BOG_matrix[27])

  (0, 17044)	0.122346591758
  (0, 16875)	0.0505344426973
  (0, 16562)	0.11856716976
  (0, 16448)	0.0876481532616
  (0, 16023)	0.114368515018
  (0, 15958)	0.111328285247
  (0, 15924)	0.109401534642
  (0, 15876)	0.09219704563
  (0, 15780)	0.10054062254
  (0, 14590)	0.0924285941421
  (0, 14324)	0.128110828987
  (0, 14225)	0.0864893830059
  (0, 14003)	0.092006694011
  (0, 13890)	0.139659678239
  (0, 13857)	0.0964611674483
  (0, 13438)	0.10145416588
  (0, 13135)	0.0891153203862
  (0, 13097)	0.123745147403
  (0, 12956)	0.104691433598
  (0, 12700)	0.108599777491
  (0, 12378)	0.0970086224212
  (0, 12181)	0.11697464841
  (0, 12173)	0.142325670723
  (0, 12162)	0.159565082391
  (0, 12151)	0.0983157920864
  :	:
  (0, 5245)	0.0841350833944
  (0, 5237)	0.113895987251
  (0, 5102)	0.11420940119
  (0, 5101)	0.184663219042
  (0, 4632)	0.043493175501
  (0, 4027)	0.0763201425025
  (0, 3957)	0.242623955266
  (0, 3875)	0.128906565576
  (0, 3822)	0.0942850049113
  (0, 3058)	0.120578921785
  (0, 3044)	0.07486

Donde los features son...

In [96]:
dict_repr = dict()
arr_repr = BOG_matrix[27].toarray()[0]
for i in BOG_matrix[27].indices:
    print('{0}: {1}'.format(representation.get_feature_names()[i], arr_repr[i]), end=', ')
    dict_repr[representation.get_feature_names()[i]] = arr_repr[i]

zona: 0.12234659175774053, yo: 0.050534442697347366, vivo: 0.11856716975995497, vi: 0.0876481532615862, una pregunta: 0.1143685150183527, una de: 0.11132828524748828, una: 0.10940153464161766, un poco: 0.09219704562998841, un: 0.10054062254010157, sobre: 0.09242859414211325, si igual: 0.1281108289871265, si: 0.0864893830059329, se si: 0.09200669401099533, se escribe: 0.13965967823906839, se: 0.09646116744828899, raro: 0.1014541658802468, que si: 0.0891153203862292, que raro: 0.12374514740338534, que los: 0.10469143359780607, que: 0.10859977749099453, pregunta: 0.09700862242124296, por donde: 0.11697464840972999, por alla: 0.14232567072328067, por: 0.15956508239073838, pongo: 0.09831579208638877, pones: 0.11587912544216117, poco: 0.085767058832565, pero es: 0.10377616506189986, pero: 0.05238518845298545, otros: 0.09759410557706343, obvio: 0.09968114551481815, no se si: 0.09274280158788244, no se: 0.06596438580282649, no no: 0.09500251833050471, no: 0.11351884900093571, mia: 0.1113282852

Palabras más representativas para la frase:

In [97]:
sorted(dict_repr.items(), key=lambda x: x[1], reverse=True)[:10]

[('la direccion', 0.26413018059094401),
 ('direccion', 0.24262395526578306),
 ('es', 0.18466321904243099),
 ('por', 0.15956508239073838),
 ('la', 0.15812530743968584),
 ('ciudades', 0.14554687618334389),
 ('aca por', 0.14554687618334389),
 ('por alla', 0.14232567072328067),
 ('chicos una', 0.14232567072328067),
 ('es un poco', 0.14049844803858233)]

Pues claramente esta pidiendo una dirección

## Word Embeddings

Tomamos algunos embeddings de:
- [SBWCE](http://crscardellino.me/SBWCE/) --> 300 dimensiones
- Nuestro corupus --> 100 dimensiones

**TWITER???**
     

#### SBWCE

Empezaremos cargando los vectores pre-entrenados con el algoritmo de Word2Vec. Para ello usaremos Gensim.

In [26]:
from gensim.models import KeyedVectors

Using TensorFlow backend.


In [3]:
model = KeyedVectors.load_word2vec_format('embeddings/SBW-vectors-300-min5.bin.gz', binary=True)

De esta forma no nos alcanza la memoria (por ahora), por lo que vamos a filtrar las palabras que usaremos y guardaremos un nuevo archivo más pequeño.

##### Achicando el vocabulario

In [12]:
from collections import Counter
from itertools import chain

In [13]:
distribution_words = Counter(chain.from_iterable(df_cleaned.map(lambda x: x.split(' '))))

In [22]:
new_word2vec = open('embeddings/new_vector_SBW.txt', 'w')

In [23]:
with open('embeddings/SBW-vectors-300-min5.txt') as wordvecs:
    for line in wordvecs:
        word = line.split(' ')
        word, features = word[0], word[1:]
        if word in distribution_words:
            new_word2vec.write(word + ' ' + ' '.join(features))

In [24]:
new_word2vec.close()

Cargando nuevamente los vectores

In [46]:
model = KeyedVectors.load_word2vec_format('embeddings/new_vector_SBW.txt', binary=False)

In [47]:
model.wv.most_similar('ok')

[('inn', 0.5931442975997925),
 ('galta', 0.5310695171356201),
 ('fikk', 0.5252362489700317),
 ('riki', 0.5229285359382629),
 ('jag', 0.5188263654708862),
 ('sista', 0.5183577537536621),
 ('jah', 0.5087394714355469),
 ('yax', 0.5061405897140503),
 ('findes', 0.5053848028182983),
 ('dane', 0.5039175748825073)]

### Corpus chat

##### Cargando el corpus

Usaremos nuestro corpus para entrenar nuestro word2vec con los siguientes parametros:

  - Minima frecuencia de palabra: 5
  - Negative samples=20

In [40]:
from gensim.models import Word2Vec

##### Guardando el modelo entrenado

In [31]:
model = Word2Vec(sentences=df_cleaned.map(lambda x: x.split(' ')), min_count=5, negative=20, workers=4)

In [32]:
model.save('embeddings/chat.w2v')

##### Cargando el modelo

In [41]:
model = Word2Vec.load('embeddings/chat.w2v')

In [45]:
model.wv.most_similar('ok')

[('dale', 0.8620980381965637),
 ('👍🏻', 0.7748069763183594),
 ('buenisimo', 0.7722575664520264),
 ('listo', 0.7669600248336792),
 ('😬', 0.7667552828788757),
 ('avisame', 0.7547341585159302),
 ('gobbi', 0.7463787198066711),
 ('dani', 0.7325387001037598),
 ('😁', 0.7173537015914917),
 ('naho', 0.7016512155532837)]

### Pero como vectorizamos un contexto??

In [50]:
import numpy as np

La primera opción fue sumar todos los vectores de palabras en un texto y así teníamos una representación del todo el contexto.

Claramente aparecía un gran problema...
- Que pasa si los contextos son similares pero tienen una longitud distinta... entonces iban a estar en espacios diferentes :o

La simple solución fue promediarlo con la longitud del contexto y perfecto
  

In [48]:
def context_to_vect(context):
    return np.array([
        np.mean([word2vec.get_vector(w)
                 for w in context if w in word2vec.get_vocab()]
                or [np.zeros(self.word2vec.get_dim())], axis=0)
        for context in df_cleaned.map(lambda x: x.split())
    ])